In [1]:
# 安装基础包
%pip install -q opencv-python-headless pillow numpy
%pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip install -q diffusers transformers accelerate
%pip install -q paddlepaddle-gpu paddleocr

# 安装 GroundingDINO 依赖（避免直接安装，使用预编译版本）
%pip install -q supervision
%pip install -q groundingdino-py  # 预编译版本，更稳定

# 安装 LaMa 依赖
%pip install -q easydict scikit-image omegaconf hydra-core
%pip install -q albumentations pytorch-lightning webdataset kornia wldhx.yadisk-direct

# 克隆仓库
!git clone -q https://github.com/advimman/lama.git /content/lama 2>/dev/null || echo "Lama already exists"

# 下载权重
!mkdir -p /content/weights

# 下载 GroundingDINO 权重
!wget -q --show-progress https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth -O /content/weights/groundingdino_swint_ogc.pth

# 下载 LaMa 模型（使用正确的下载方式）
!mkdir -p /content/weights/lama
!wget -q --show-progress "https://github.com/advimman/lama/releases/download/0.1.0/big-lama.zip" -O /content/weights/lama/big-lama.zip
!unzip -q /content/weights/lama/big-lama.zip -d /content/weights/lama/ 2>/dev/null || echo "Extracting..."

# SD Inpaint 模型（使用公开模型）
%pip install -q huggingface-hub
# 使用公开的 SD Inpaint 模型，不需要认证
from huggingface_hub import hf_hub_download
import os
os.makedirs("/content/weights/sd_inpaint", exist_ok=True)

# 使用 Stable Diffusion Inpaint 基础模型（公开可用）
print("Models will be downloaded on first use from runwayml/stable-diffusion-inpainting")

print("✅ 安装完成！")




[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone may have tampered with them.
    transformers from https://files.pythonhosted.org/packages/6a/6b/2f416568b3c4c91c96e5a365d164f8a4a4a88030aa8ab4644181fdadce97/transformers-4.57.3-py3-none-any.whl:
        Expected sha256 c77d353a4851b1880191603d36acb313411d3577f6e2897814f333841f7003f4
             Got        ed56ae45b3dc0007174a916a76d6431a1c4b5fd55540d4d3565a465f754bbfd1


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice]

/Users/george/Documents/me/skiing/showme/skiing/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: [Errno 30] Read-only file system: '/content'

In [ ]:
import sys
import os
sys.path.insert(0, '/content/lama')  # LaMa 路径

import cv2
import torch
import numpy as np
from PIL import Image
from paddleocr import PaddleOCR

# 导入 GroundingDINO（使用简化的 API）
try:
    from groundingdino.util.inference import Model as GroundingDINOModel
    GDINO_AVAILABLE = True
except:
    print("⚠️ GroundingDINO 不可用，将仅使用 OCR 检测")
    GDINO_AVAILABLE = False

# 导入 diffusers
from diffusers import StableDiffusionInpaintPipeline

# 初始化设备
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ 使用设备: {device}")

# 初始化 PaddleOCR
print("📝 初始化 PaddleOCR...")
ocr = PaddleOCR(
    use_angle_cls=True, 
    lang="en",  # 使用英文识别（水印通常是英文）
    use_gpu=True if device=="cuda" else False, 
    det_db_thresh=0.3,
    show_log=False
)

# 初始化 GroundingDINO（如果可用）
gdino_model = None
if GDINO_AVAILABLE:
    try:
        print("🔍 初始化 GroundingDINO...")
        CONFIG_PATH = "/content/weights/groundingdino_swint_ogc.pth"
        if os.path.exists(CONFIG_PATH):
            gdino_model = GroundingDINOModel(
                model_config_path="GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py",
                model_checkpoint_path=CONFIG_PATH,
                device=device
            )
            print("✅ GroundingDINO 加载成功")
        else:
            print("⚠️ GroundingDINO 权重未找到，跳过")
    except Exception as e:
        print(f"⚠️ GroundingDINO 加载失败: {e}")
        gdino_model = None

# 初始化 LaMa（使用简单的 inpainting）
print("🎨 准备 Inpainting 模型...")
lama_available = False
try:
    sys.path.insert(0, '/content/lama')
    import yaml
    from omegaconf import OmegaConf
    # 简化的 LaMa 加载
    lama_path = "/content/weights/lama/big-lama"
    if os.path.exists(lama_path):
        lama_available = True
        print("✅ LaMa 模型可用")
except Exception as e:
    print(f"⚠️ LaMa 加载失败: {e}")

# 初始化 Stable Diffusion Inpaint Pipeline
print("🖼️ 初始化 Stable Diffusion Inpainting...")
try:
    pipe = StableDiffusionInpaintPipeline.from_pretrained(
        "runwayml/stable-diffusion-inpainting",
        torch_dtype=torch.float16 if device=="cuda" else torch.float32,
        safety_checker=None
    ).to(device)
    
    # 启用内存优化
    if device == "cuda":
        pipe.enable_xformers_memory_efficient_attention()
        pipe.enable_attention_slicing()
    
    print("✅ SD Inpainting 加载成功")
except Exception as e:
    print(f"❌ SD Inpainting 加载失败: {e}")
    pipe = None

print("\n🎉 模型初始化完成！")

In [ ]:
def merge_masks(mask1, mask2):
    """合并两个 mask"""
    return np.clip(mask1 + mask2, 0, 255).astype(np.uint8)

def simple_inpaint(img, mask):
    """简单的 inpainting 使用 OpenCV"""
    return cv2.inpaint(img, mask, 3, cv2.INPAINT_TELEA)

def remove_watermark(image_path, output_path, use_sd=True, debug=False):
    """
    移除图片水印
    
    Args:
        image_path: 输入图片路径
        output_path: 输出图片路径
        use_sd: 是否使用 Stable Diffusion（否则仅用 OpenCV）
        debug: 是否保存调试图片
    """
    print(f"📷 处理图片: {image_path}")
    
    # 读取图片
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ 无法读取图片: {image_path}")
        return
    
    h, w = img.shape[:2]
    print(f"   尺寸: {w}x{h}")
    
    # Step 1: PaddleOCR 检测文本
    print("📝 Step 1: OCR 检测文本...")
    ocr_result = ocr.ocr(image_path, cls=True, det=True, rec=False)
    ocr_mask = np.zeros((h, w), dtype=np.uint8)
    
    if ocr_result and ocr_result[0]:
        for line in ocr_result[0]:
            box = np.array(line[0], dtype=np.int32)
            cv2.fillPoly(ocr_mask, [box], 255)
        print(f"   ✅ 检测到 {len(ocr_result[0])} 个文本区域")
    else:
        print("   ⚠️ 未检测到文本")
    
    # 膨胀 mask 以覆盖更多区域
    if np.sum(ocr_mask) > 0:
        ocr_mask = cv2.dilate(ocr_mask, np.ones((15,15), np.uint8), iterations=2)
    
    # Step 2: GroundingDINO 补全（如果可用）
    gdino_mask = np.zeros((h, w), dtype=np.uint8)
    if gdino_model is not None:
        try:
            print("🔍 Step 2: GroundingDINO 检测...")
            detections = gdino_model.predict_with_classes(
                image=img,
                classes=["watermark", "text", "logo", "signature"],
                box_threshold=0.25,
                text_threshold=0.20
            )
            
            if len(detections) > 0:
                for box in detections.xyxy:
                    x1, y1, x2, y2 = map(int, box)
                    cv2.rectangle(gdino_mask, (x1, y1), (x2, y2), 255, -1)
                print(f"   ✅ 检测到 {len(detections)} 个目标")
            else:
                print("   ⚠️ 未检测到目标")
        except Exception as e:
            print(f"   ⚠️ GroundingDINO 检测失败: {e}")
    else:
        print("⏭️  Step 2: 跳过 GroundingDINO（不可用）")
    
    # Step 3: 合并 mask
    print("🎭 Step 3: 合并检测结果...")
    final_mask = merge_masks(ocr_mask, gdino_mask)
    
    if np.sum(final_mask) == 0:
        print("⚠️ 未检测到任何水印区域，跳过处理")
        # 直接复制原图
        import shutil
        shutil.copy(image_path, output_path)
        return
    
    # 再次膨胀以确保完全覆盖
    final_mask = cv2.dilate(final_mask, np.ones((21,21), np.uint8), iterations=3)
    
    # 保存调试图片
    if debug:
        debug_dir = os.path.dirname(output_path)
        base_name = os.path.splitext(os.path.basename(output_path))[0]
        cv2.imwrite(f"{debug_dir}/{base_name}_mask.png", final_mask)
        print(f"   💾 保存 mask: {debug_dir}/{base_name}_mask.png")
    
    # Step 4: Inpainting
    if use_sd and pipe is not None:
        print("🎨 Step 4: Stable Diffusion Inpainting...")
        try:
            img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            mask_pil = Image.fromarray(final_mask).convert('L')
            
            result = pipe(
                prompt="natural background, seamless, high quality",
                negative_prompt="watermark, text, logo, signature, blurry, low quality, distorted",
                image=img_pil,
                mask_image=mask_pil,
                strength=0.5,
                guidance_scale=7.5,
                num_inference_steps=30
            ).images[0]
            
            result.save(output_path)
            print(f"✅ SD Inpainting 完成")
        except Exception as e:
            print(f"❌ SD Inpainting 失败: {e}")
            print("   使用 OpenCV 备用方案...")
            result = simple_inpaint(img, final_mask)
            cv2.imwrite(output_path, result)
    else:
        print("🎨 Step 4: OpenCV Inpainting...")
        result = simple_inpaint(img, final_mask)
        cv2.imwrite(output_path, result)
    
    print(f"✅ 处理完成，已保存: {output_path}\n")

# 测试函数
def test_on_sample():
    """在示例图片上测试"""
    test_image = "/content/test_watermark.jpg"
    output_image = "/content/test_output.jpg"
    
    # 检查测试图片是否存在
    if not os.path.exists(test_image):
        print(f"⚠️ 测试图片不存在: {test_image}")
        print("请上传一张带水印的图片到 /content/test_watermark.jpg")
        return
    
    remove_watermark(test_image, output_image, use_sd=True, debug=True)
    
    # 显示结果
    try:
        from IPython.display import Image as IPImage, display
        print("\n原图:")
        display(IPImage(filename=test_image))
        print("\n处理后:")
        display(IPImage(filename=output_image))
    except:
        print("在非 notebook 环境中，请手动查看输出图片")

print("✅ 函数定义完成！")
print("\n使用方法:")
print("1. 上传图片到 /content/test_watermark.jpg")
print("2. 运行: test_on_sample()")
print("3. 或直接调用: remove_watermark(input_path, output_path)")


remove_watermark(
    "https://imgalit.fenxuekeji.com/prod/photo_wall/96/2025-11-23/photo/73fedd28-ba12-1152446659842846720_x700.JPG", 
    "/content/output.jpg",
    use_sd=True,  # 使用 SD Inpainting
    debug=True    # 保存调试信息
)
